In [1]:
from IPython.core.display import display, HTML
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
url_pattern = 'https://myanimelist.net/topanime.php' 
query_param_pattern = 'limit'

def assembly_url(page=0):
    return "{}?{}={}".format(url_pattern, query_param_pattern, page * 50) if page > 0 else url_pattern

def process_item(item_node):
    link = item_node.select("a[id^='#area']:only-child")[0]
    return {
        'id': int(re.search(r'\d+$', link.get('id')).group(0)),
        'name': link.contents[0],
        'url': link.get('href'),
        'score': float(item_node.select('span.on.score-label')[0].contents[0]),
        'position': int(item_node.select('span.top-anime-rank-text')[0].contents[0])
    }
  
    

def process_toplist(page=0):
    result = []
    url = assembly_url(page)
    req = requests.get(url)
    
    if req.status_code == 200:
        parser = BeautifulSoup(req.content, 'html.parser')
        items = parser.select('tr.ranking-list')
        for item in items:
            result.append(process_item(item))
    else:
        print('trying to access url {} resulted in HTTP error {}'.format(url, req.status_code))
        
    return result

In [3]:
def test_get_best_100():
    results = [ item for items in (process_toplist(i) for i in range(2)) for item in items ]
    table = pd.DataFrame(results);
    display(HTML(table.to_html()))

In [4]:
test_get_best_100()

,id,name,url,score,position
0,5114,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood,9.22,1
1,9253,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.13,2
2,11061,Hunter x Hunter (2011),https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011,9.12,3
3,28977,Gintama°,https://myanimelist.net/anime/28977/Gintama°,9.11,4
4,820,Ginga Eiyuu Densetsu,https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu,9.10,5
5,9969,Gintama',https://myanimelist.net/anime/9969/Gintama,9.09,6
6,38524,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2,9.07,7
7,15417,Gintama': Enchousen,https://myanimelist.net/anime/15417/Gintama__Enchousen,9.04,8
8,35180,3-gatsu no Lion 2nd Season,https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season,9.02,9
9,32281,Kimi no Na wa.,https://myanimelist.net/anime/32281/Kimi_no_Na_wa,9.01,10
